In [1]:
import json
from itertools import chain

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm_notebook as tqdm


import findspark
findspark.init()

import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

from itertools import product

In [9]:
pd.set_option('display.max_colwidth', -1)

In [2]:
test_f = pd.read_json("./data/test_files.json")
train_f = pd.read_json("./data/train_files.json")
dev_f = pd.read_json("./data/dev_files.json")
print(test_f.columns)
test_f.head()

Index(['phase', 'table_id', 'question', 'sql', 'answer', 'sql_query'], dtype='object')


,phase,table_id,question,sql,answer,sql_query
0,1,1-10015132-16,What is terrence ross' nationality,"{'sel': 2, 'conds': [[0, 0, 'Terrence Ross']],...",[united states],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', '..."
1,1,1-10015132-16,What clu was in toronto 1995-96,"{'sel': 5, 'conds': [[4, 0, '1995-96']], 'agg'...",[arkansas],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', '..."
2,1,1-10015132-16,which club was in toronto 2003-06,"{'sel': 5, 'conds': [[4, 0, '2003-06']], 'agg'...",[michigan],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', '..."
3,1,1-10015132-16,how many schools or teams had jalen rose,"{'sel': 5, 'conds': [[0, 0, 'Jalen Rose']], 'a...",[1],"{'agg_index': 3, 'cond_ops': ['=', '>', '<', '..."
4,1,1-10083598-1,Where was Assen held?,"{'sel': 2, 'conds': [[3, 0, 'Assen']], 'agg': 0}",[netherlands],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', '..."


In [3]:
test_tb = spark.read.json("./data/test_tables.jsonl").toPandas().rename(columns={'id':'table_id'})
train_tb = spark.read.json("./data/train_tables.jsonl").toPandas().rename(columns={'id':'table_id'})
dev_tb = spark.read.json("./data/dev_tables.jsonl").toPandas().rename(columns={'id':'table_id'})
test_tb.loc[0]

caption                                                          R
header           [Player, No., Nationality, Position, Years in ...
table_id                                             1-10015132-16
name                                             table_10015132_16
page_id                                                        NaN
page_title                         Toronto Raptors all-time roster
rows             [[Aleksandar Radojević, 25, Serbia, Center, 19...
section_title                                                    R
types                         [text, text, text, text, text, text]
Name: 0, dtype: object

In [4]:
def header_trans(df):
    header_list = []
    for i, row in df.iterrows():
        header_new = list(zip(row['header'], row['types'],row['rows'][0]))
        header_list.append(header_new) 
    df['header_new']=header_list
    df = df.drop(columns=['header'])
    return df

In [5]:
test_tb= header_trans(test_tb)
train_tb= header_trans(train_tb)
dev_tb= header_trans(dev_tb)

In [6]:
test_tb.loc[0]

caption                                                          R
table_id                                             1-10015132-16
name                                             table_10015132_16
page_id                                                        NaN
page_title                         Toronto Raptors all-time roster
rows             [[Aleksandar Radojević, 25, Serbia, Center, 19...
section_title                                                    R
types                         [text, text, text, text, text, text]
header_new       [(Player, text, Aleksandar Radojević), (No., t...
Name: 0, dtype: object

In [7]:
test = pd.merge(test_f,test_tb,on=['table_id']).rename(columns={'question':'context','page_title':'title','header_new':'header'})
train = pd.merge(train_f,train_tb,on=['table_id']).rename(columns={'question':'context','page_title':'title','header_new':'header'})
dev = pd.merge(dev_f,dev_tb,on=['table_id']).rename(columns={'question':'context','page_title':'title','header_new':'header'})

In [10]:
test.iloc[1]

phase            1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
table_id         1-10015132-16                                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
test.to_json('./data/test_tabert.json')
train.to_json('./data/train_tabert.json')
dev.to_json('./data/dev_tabert.json')